## Hashmaps

1. Hashmaps represents a map from a set of keys to a set of value.
    * They are a key-value storage with fast assignment and lookup.
    * Built on top of an array using special indexing.


2. A hash function is used to convert the key into a hash code, which is then convered (for example, hash code % size) to array index at which the value is stored. `Key -> Hash Function -> Hash Code -> Hash Code % size -> array index -> assign value to that index`
    * A hash function can take string or any type of data as input and produces a hash code as output.
    * Example of a simple hash function -> `count of 'a' + count of 'e'` in the input key
    * Different keys can result in the same hash value. Thus hashing is a non-reversible process, you cannot get the key from the hash code.
    * The hash function needs to be computationally simple, since hashing is performed everytime you assign or retrieve a value from a hashmap.


3. Collisions - When hash function produces the same hash value for different keys. There are several strategies to resolve hash collisions.
    * **Separate Chaining** - Avoids collisions by updating the underlying datastructure from array of values to array of linked lists.
        * If the value at the array index is empty - Create new linked list at that index with first element as the key's value.
        * If there is already a linked list at the array index - Append the value to that linked list.
        * You also need to save the key along with the value, this is required when you retrieve the value for a given key. 
        `Key -> Hash Code -> array index -> Iterate over the linked list at the index to find the value with matching key`
        * Separate chaining is effective for hash functions that are particularly good at giving unique indices so that the linked list never gets very long.
        
    * **Open Addressing** - We stick to the array as the underlying data structure. When a collision is found, we continue looking for a new index until an empty index is found to save our data.
        * One of the methods of open addressing is Linear Probing. Here, we linearing update the array index till an empty index if found.
        * We also save the key which helps during retrieval. When looking up for a value, check if the key at the resulting index matches your key. If yes, then return it. If no, then keep probing (move to the next index) till you find your matching key.
        * There are other complicated methods for open addressing as well, for example, quadratic probing, etc.

## Creating a Hashmap class

Let's create a Hashmap class in Python that uses Open Addressing for collision resolution.

### Hashmap using Open Adressing

The underlying data structure is an array. We will be using a Python list to simualte the underlying array for this hashmap class.

In [1]:
class Hashmap:
    def __init__(self, array_size):
        self.array_size = array_size
        self.array = [None for i in range(array_size)]
    
    def hash(self, key, count_collisions=0):
        # convert key to a list of bytes
        key_bytes = key.encode()
        # convert the list of bytes to a hash code
        hash_code = sum(key_bytes)
        return hash_code+count_collisions
    
    def compressor(self, hash_code):
        return hash_code % self.array_size
    
    def assign(self, key, value):
        array_index = self.compressor(self.hash(key))
        current_array_value = self.array[array_index]
        if current_array_value == None:
            self.array[array_index] = [key, value]
            return
        
        if current_array_value[0] == key:
            self.array[array_index] = [key, value]
            return
        
        if current_array_value[0] != key:
            number_collisions = 1
            while current_array_value[0] != key:
                new_hash_code = self.hash(key, number_collisions)
                new_array_index = self.compressor(new_hash_code)
                current_array_value = self.array[new_array_index]
                if current_array_value == None:
                    self.array[new_array_index] = [key, value]
                    return
                if current_array_value[0] == key:
                    self.array[new_array_index] = [key, value]
                    return
                if current_array_value[0] != key:
                    number_collisions += 1
            
    
    def retrieve(self, key):
        array_index = self.compressor(self.hash(key))
        possible_return_value = self.array[array_index]
        if possible_return_value == None:
            return None
        if possible_return_value[0] == key:
            return possible_return_value[1]
        if possible_return_value[0] != key:
            retrieval_collisions = 1
            while possible_return_value[0] != key:
                new_hash_code = self.hash(key, retrieval_collisions)
                retrieving_array_index = self.compressor(new_hash_code)
                possible_return_value = self.array[retrieving_array_index]
                if possible_return_value == None:
                    return None
                if possible_return_value[0] == key:
                    return possible_return_value[1]
                if possible_return_value[0] != key:
                    retrieval_collisions += 1
        

In [2]:
a = Hashmap(15)

In [3]:
a.assign("Virat Kohli", "India")
a.assign("Steve Smith", "Australia")
a.assign("Kane Williamson", "New Zealand")
a.assign("Joe Root", "England")

In [4]:
a.retrieve("Virat Kohli")

'India'

In [5]:
a.retrieve("Joe Root")

'England'

### Hashmap using Separate Chaining

First, let's add Node and LinkedList classes.

In [6]:
class Node:
    def __init__(self, value, next_node=None):
        self.value = value
        self.next_node = next_node
    
    def get_value(self):
        return self.value
    
    def get_next_node(self):
        return self.next_node
    
    def set_next_node(self, next_node):
        self.next_node = next_node

In [7]:
class LinkedList:
    def __init__(self, head_node=None):
        self.head_node = head_node
  
    def insert(self, new_node):
        current_node = self.head_node
        if not current_node:
            self.head_node = new_node
        while(current_node):
            next_node = current_node.get_next_node()
            if not next_node:
                current_node.set_next_node(new_node)
            current_node = next_node

    def __iter__(self):
        current_node = self.head_node
        while(current_node):
            yield current_node.get_value()
            current_node = current_node.get_next_node()

Let's create a Hashmap class in Python that uses Separate Chaining for collision resolution.

In [8]:
class Hashmap2:
    def __init__(self, size):
        self.size = size
        self.array = [LinkedList() for i in range(size)]
    
    def hash(self, key):
        # convert key to a list of bytes
        key_bytes = key.encode()
        # convert the list of bytes to a hash code
        hash_code = sum(key_bytes)
        return hash_code
    
    def compressor(self, hash_code):
        return hash_code % self.size
    
    def assign(self, key, value):
        array_index = self.compressor(self.hash(key))
        payload = Node([key, value])
        list_at_array = self.array[array_index]
        for item in list_at_array:
            if key == item[0]:
                item[1] = value
                return
        # if the key doesn't exist then insert a node for the key-value pair 
        list_at_array.insert(payload)
        
    def retrieve(self, key):
        array_index = self.compressor(self.hash(key))
        list_at_array = self.array[array_index]
        for item in list_at_array:
            if key == item[0]:
                return item[1]
        
        return

In [9]:
b = Hashmap2(15)

In [10]:
b.assign("Virat Kohli", "India")
b.assign("Steve Smith", "Australia")
b.assign("Kane Williamson", "New Zealand")
b.assign("Joe Root", "England")

In [11]:
b.retrieve("Virat Kohli")

'India'

In [12]:
b.retrieve("Joe Root")

'England'